![HPEDEVLogo](Pictures/hpe-dev-logo.png)

# Welcome to the {{ BRANDINGWOD }} Hack Shack
[{{ BRANDING }} Community Team]({{ BRANDINGURL }})

# Instructor: [Alvin Castro](mailto:aruba-automation@hpe.com)

<p align="center">
  <img src="Pictures/hackshackdisco.png">
  
</p>

# Introduction to the pyaoscx v2.0 Aruba Python SDK
This lab is an introduction to Aruba's Python SDK, named pyaoscx v2.0.  Note that this instruction will be specific to the v2.0 or later pyaoscx module, as the class structure and method calls all function differently compared to v1.x

# Prerequisites
Prior to this lab, users must have a basic understanding of Python v3.x as well as fundamentals of class structures and basic programming concepts.
Some helpful resources to getting started with Python concepts can be found in other HPE Hackshack courses, such as:
* [Introduction to Python Programming](https://hackshack.hpedev.io/workshop/15)
* [API 101 - API basics and the value they provide](https://hackshack.hpedev.io/workshop/9)

# Lab Flow
{{ BRANDINGWOD }} Workshops-on-Demand are delivered through a central point that allows a portable, dynamic version of the lab guides. We use a JupyterHub server on which all the different labs guides are stored in a notebook format (*.ipynb). These Notebooks are accessible from the internet for the event.

The notebooks can be downloaded on to your own laptop for further use. In order to use  them, you will need to install the Jupyter Notebook application available [here](https://jupyter.org/install).
Beginners guide available [here](https://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)

Please now listen to the instructor guidelines on how to use this notebook and follow the different steps.

Happy labs ! :-)



## Task 1 Setup a Python workflow skeleton to login to the switch

In this task we are going to create a Python workflow skeleton that will basically login and logout of the switch.  This will function as the basic framework for further workflows that will connect using the pyaoscx v2 SDK.

* The pyaoscx SDK can be publicly downloaded on your own workstation [here](https://github.com/aruba/pyaoscx) 
* The Jupyter Notebook already has the pyaoscx v2.x SDK installed.
* The pyaoscx v2 SDK interacts with Aruba switches via APIs

### Task 1.1 Setting Imports for skeleton

* First we are going to import necessary used libraries for our later function calls
* We are going to use the 'from x import y' for the imports to help identify which function calls are coming from which particular library.

In [ ]:
#### 1.1 Example with notes

### Importing urllib3 to turn off any warnings regarding connections through HTTPS in this enclosed environment
import urllib3
### Session will be imported as a means to login and logout of the switch
from pyaoscx.session import Session
### PyaoscxFactory will be used as the vehicle to update all of the necessary attributes in a JSON to the switch
from pyaoscx.pyaoscx_factory import PyaoscxFactory
### VLAN will be imported to update and make changes to VLANs
from pyaoscx.vlan import Vlan
### Interface will be imported to update and make changes to interfaces (physical and logical)
from pyaoscx.interface import Interface

### This is to turn off the warnings
urllib3.disable_warnings()

### Task 1.2 Setting Session information to Log in and Log out of the Switch

* Next we will set parameters necessary to create a Session object that will be used to login to the Switch
* Once we Open a session to login, we will set a 'try' block to run a couple of print statements for output
* In a 'finally' block we will logout of the switch

We will be using the Session class to run the 'open' function to open a session that will be used to login into the switch.  This will require specifying the API version that the pyaoscx library will use, as well as the switch's IP address and credentials to login to the switch.

Note that for the functions that will be affecting the switch configuration, we are opting to use a 'try' block with 'except' and 'finally'.  The 'try' block will run the actual workflow, and but will exit out of the block upon errors (passing those to the exception).  The 'finally' block will always run, no matter whether the 'try' block as exited or fully completed its workflow.  The reason is to allow the script to catch any errors and exceptions, while still forcing the script to logout in the 'finally' block.

**It is important to end your workflows with a logout in the finally section, to prevent the switch from having too many open sessions, which can lock you out unless you either wait for the sessions to timeout or run the command to close all sessions.**

In [ ]:
#### 1.2 Example with notes

import urllib3
from pyaoscx.session import Session
from pyaoscx.pyaoscx_factory import PyaoscxFactory
from pyaoscx.vlan import Vlan
from pyaoscx.interface import Interface

urllib3.disable_warnings()

# Set variables for creating the Session object
version = '10.04'  # Version of API that this will be using.
switch_ip = '16.31.85.161'  #  Switch IP Address to connect to
username = 'admin' # Username for admin account to login to the switch
password = 'Arubacx$' # Password for admin account to login to the switch

# Create Session object
s = Session(switch_ip, version)
# Run Session.open() method to login to the switch with the credentials
s.open(username, password)
print("Logging into the switch at ", switch_ip)
# Try block is used so that session closes even on error
try:
    # Insert some print statements to see an output
    print("By this point you should now be logged into the switch")
    print("Hello World")

# Except block is used to catch any errors and print them out.  This will be useful as we make complex workflows going forward.
except Exception as error:
        print('Ran into exception: {}.  Closing session.'.format(error))
        
# Finally block always runs, no matter whether of not the try block fully completed.
finally:
    # Close the session to logout of the switch
    print("Now logging out of the switch.")
    s.close()
    

---
If all is correct, you should see something like the following:

```
Logging into the switch at 172.168.23.100
By this point you should now be logged into the switch.
Hello World
Now logging out of the switch.
```

* If you are successful first time, why not try to run the code again but with the wrong username or password?  

## Task 2 Open Granulated Approach vs Imperative Factory Approach

The way pyaoscx v2 is structured allows for two different ways to create and configure the settings with the switch API through the use of objects.
In the Open Granulated Approach, we are focusing on the objects that we want to change, such as the VLAN or the interface.
In the Imperative Factory Approach, we are looking at the switch configuration as a whole, through the use of the concept of an object factory.

This section will give examples of both the Open Granulated Approach as well as the Imperative Factory Approach, to allow users to find which workflow style is more comfortable for them.

### Task 2.1 Open Granulated Approach to Create an Object

* We will Create a VLAN object and validate that it was created using the Open Granulated Approach

*Some notes and print output was left in order to help with debugging and remind of it's use*

The VLAN object constructor has parameters that require the session, the vlan ID, and any optional parameters that can be passed as keyword arguments.  More information of this class can be found [here](https://github.com/aruba/pyaoscx/blob/master/pyaoscx/vlan.py)

**For this upcoming exercise, fill in the 'vlan100=' line using the constructor with parameters Vlan(*session, vlanID, name, voice*).**

* *session* is for the variable holding the session data.
* *vlanID* is an integer for the VLAN ID number you're creating.
* *name* is a String for the name of the VLAN. Don't forget to use quotes around the whole String, such as "VLAN 100".
* *voice* is a Boolean (True or False) to determine if the VLAN is a Voice VLAN or not.

The code section afterwards has an example answer.

*Note that if you're getting a 500 error when running the code, there may already be a VLAN 100 already on that switch.  For this task, feel free to change the VLAN to another number if that is the error you are getting.  The next section will show how to delete a VLAN.

In [ ]:
#### 2.1 VLAN Create Example with notes

import urllib3
from pyaoscx.session import Session
from pyaoscx.pyaoscx_factory import PyaoscxFactory
from pyaoscx.vlan import Vlan
from pyaoscx.interface import Interface

urllib3.disable_warnings()

# Set variables for creating the Session object
version = '10.04'
switch_ip = '16.31.85.161'
username = 'admin'
password = 'Arubacx$'

# Create Session object
s = Session(switch_ip, version)
s.open(username, password)
print("Logging into the switch at ", switch_ip)

try:
    print("By this point you should now be logged into the switch")

    # Create a VLAN object.  Note that at this point the object is created on the Python instance, and is not yet pushed to the switch
    vlan100 = 
    # This VLAN object is passing in the required parameters for the Session object, the VLAN ID of 100, and the optional parameters 
    # for setting the name to "VLAN 100" as well as setting it as a voice VLAN
    
    # Since the VLAN object has not been "pushed" to the switch, the apply() method is used to perform an API POST request
    vlan100.apply()
    
    
except Exception as error:
        print('Ran into exception: {}.  Closing session.'.format(error))
        
finally:
    print("Now logging out of the switch.")
    s.close()
    

In [ ]:
#### 2.1 VLAN Create Answers Example with notes

import urllib3
from pyaoscx.session import Session
from pyaoscx.pyaoscx_factory import PyaoscxFactory
from pyaoscx.vlan import Vlan
from pyaoscx.interface import Interface

urllib3.disable_warnings()

# Set variables for creating the Session object
version = '10.04'
switch_ip = '16.31.85.161'
username = 'admin'
password = 'Arubacx$'

# Create Session object
s = Session(switch_ip, version)
s.open(username, password)
print("Logging into the switch at ", switch_ip)

try:
    print("By this point you should now be logged into the switch")

    # Create a VLAN object.  Note that at this point the object is created on the Python instance, and is not yet pushed to the switch
    vlan100 = Vlan(s, 100, name="VLAN 100", voice=True)
    # This VLAN object is passing in the required parameters for the Session object, the VLAN ID of 100, and the optional parameters 
    # for setting the name to "VLAN 100" as well as setting it as a voice VLAN
    
    # Since the VLAN object has not been "pushed" to the switch, the apply() method is used to perform an API POST request
    vlan100.apply()
    
    
except Exception as error:
        print('Ran into exception: {}.  Closing session.'.format(error))
        
finally:
    print("Now logging out of the switch.")
    s.close()
    

---
You may notice that your output will look similar to:

```
Logging into the switch at 172.168.23.100
By this point you should now be logged into the switch.
Now logging out of the switch.
```

There isn't really any mention of the VLAN changes, because we did not print anything out.  When running an apply() method, the return is a boolean True when successful, or a False if there were any exceptions raised.  If you have access to the switch, you could check a switch's CLI (or via API) to verify that the VLAN 100 was created successfully.  For this workshop, we will use the Vlan.get_all() function to print out the VLANs currently on the switch and use that to validate which VLANs are currently there.

In [ ]:
#### 2.1 Get All VLANs Example with notes

import urllib3
from pyaoscx.session import Session
from pyaoscx.pyaoscx_factory import PyaoscxFactory
from pyaoscx.vlan import Vlan
from pyaoscx.interface import Interface

urllib3.disable_warnings()

# Set variables for creating the Session object
version = '10.04'
switch_ip = '16.31.85.161'
username = 'admin'
password = 'Arubacx$'

# Create Session object
s = Session(switch_ip, version)
s.open(username, password)
print("Logging into the switch at ", switch_ip)

try:
    print("By this point you should now be logged into the switch")

    # Getting a list of all VLAN objects on the switch
    print("Checking the switch for a list of all VLANs")
    all_vlans = Vlan.get_all(s)
    print(all_vlans.keys())    
    
except Exception as error:
        print('Ran into exception: {}.  Closing session.'.format(error))
        
finally:
    print("Now logging out of the switch.")
    s.close()
    

---
You may notice that your output will look similar to:

```
Logging into the switch at 172.168.23.100
By this point you should now be logged into the switch
Checking the switch for a list of all VLANs
dict_keys([1, 10, 100, 1000, 11, 12, 13, 2, 20, 200, 300, 3000, 4, 400, 6001, 6002, 6003, 6004, 6005, 6006])
Now logging out of the switch.
```
The Vlan.get_all() function saves the VLAN data as a dictionary, where each key is the VLAN ID and the value for each key is the reference to the VLAN object data.
Since the VLAN ID is all that is necessary for validation, the .keys() function was used to selectively print that information.

Now that we have a way to retrieve all of the VLANs in the switch, logic and conditionals can be used to create output based on that information.  This next exercise creates a block of code that will check if the VLAN 100 is already created on the switch.  If the VLAN already exists, we will delete it prior to running the create() method.  In order to delete a VLAN, a VLAN object must first be created in the Python script, then the delete() method will need to be run on that object to connect to the switch and remove it.

1. Fill in the logic inside of the if() condition
2. In that if block, create the VLAN object after vlanX= for VLAN 100 to delete it from the switch.
3. Otherwise, print a line saying **VLAN 100 does not exist on the switch.**

*Without logic to handle existing VLANs, if the API tries to add a VLAN that already exists on the switch, it will return with a 500 error.*

In [ ]:
#### 2.1 Delete VLAN Example with notes

import urllib3
from pyaoscx.session import Session
from pyaoscx.pyaoscx_factory import PyaoscxFactory
from pyaoscx.vlan import Vlan
from pyaoscx.interface import Interface

urllib3.disable_warnings()

# Set variables for creating the Session object
version = '10.04'
switch_ip = '16.31.85.161'
username = 'admin'
password = 'Arubacx$'

# Create Session object
s = Session(switch_ip, version)
s.open(username, password)
print("Logging into the switch at ", switch_ip)

try:
    print("By this point you should now be logged into the switch")

    # Getting a list of all VLAN objects on the switch
    print("Checking the switch for a list of all VLANs")
    all_vlans = Vlan.get_all(s)
    print(all_vlans.keys())
    # Checking if VLAN 100 is in the list of current VLANs, then removing it if True
    if ():
        print("VLAN 100 already exists, removing it to continue.")
        # Creating a VLAN object with the ID of 100, then running the delete() method on it
        vlanX =
        vlanX.delete()

        all_vlans = Vlan.get_all(s)
        print(all_vlans.keys())
    
    else:        
    
    
except Exception as error:
        print('Ran into exception: {}.  Closing session.'.format(error))
        
finally:
    print("Now logging out of the switch.")
    s.close()
    

In [ ]:
#### 2.1 Delete VLAN Answer Example with notes

import urllib3
from pyaoscx.session import Session
from pyaoscx.pyaoscx_factory import PyaoscxFactory
from pyaoscx.vlan import Vlan
from pyaoscx.interface import Interface

urllib3.disable_warnings()

# Set variables for creating the Session object
version = '10.04'
switch_ip = '16.31.85.161'
username = 'admin'
password = 'Arubacx$'

# Create Session object
s = Session(switch_ip, version)
s.open(username, password)
print("Logging into the switch at ", switch_ip)

try:
    print("By this point you should now be logged into the switch")

    # Getting a list of all VLAN objects on the switch
    print("Checking the switch for a list of all VLANs")
    all_vlans = Vlan.get_all(s)
    print(all_vlans.keys())
    # Checking if VLAN 100 is in the list of current VLANs, then removing it if True
    if (100 in all_vlans):
        print("VLAN 100 already exists, removing it to continue.")
        # Creating a VLAN object with the ID of 100, then running the delete() method on it
        vlanX = Vlan(s, 100)
        vlanX.delete()
            
        all_vlans = Vlan.get_all(s)
        print(all_vlans.keys())
    
    else:
        print("VLAN 100 does not exist on the switch.")
    
    
except Exception as error:
        print('Ran into exception: {}.  Closing session.'.format(error))
        
finally:
    print("Now logging out of the switch.")
    s.close()
    

---
Now your output should something similar to:

```
Logging into the switch at 172.168.23.100
By this point you should now be logged into the switch
Checking the switch for a list of all VLANs
dict_keys([1, 10, 100, 1000, 11, 12, 13, 2, 20, 200, 300, 3000, 4, 400, 6001, 6002, 6003, 6004, 6005, 6006])
VLAN 100 already exists, removing it to continue.
dict_keys([1, 10, 1000, 11, 12, 13, 2, 20, 200, 300, 3000, 4, 400, 6001, 6002, 6003, 6004, 6005, 6006])
Now logging out of the switch.
```


### Task 2.2 Open Granulated Approach to Update an Object

* We will make changes to a VLAN object and validate those changes using the Open Granulated Approach

With the Open Granulated Approach, users have the capability to retrieve the settings of an existing VLAN object and modify the attributes of the object directly.  Earlier the VLAN 100 was created as a Voice VLAN optionally.  The code below has a skeleton with the previous exercises rolled together.
In this next task: 
1. Change the VLAN voice attribute to False.
2. Create a temporary VLAN object (vlanx) for VLAN 100 and use the VLAN get() method to retrieve the data of that VLAN.
3. Print out the vlanx voice attribute.

The VLAN voice attribute is named *voice*


In [ ]:
#### 2.1 VLAN Update Example with notes

import urllib3
from pyaoscx.session import Session
from pyaoscx.pyaoscx_factory import PyaoscxFactory
from pyaoscx.vlan import Vlan
from pyaoscx.interface import Interface

urllib3.disable_warnings()

# Set variables for creating the Session object
version = '10.04'
switch_ip = '16.31.85.161'
username = 'admin'
password = 'Arubacx$'

# Create Session object
s = Session(switch_ip, version)
s.open(username, password)
print("Logging into the switch at ", switch_ip)

try:
    print("By this point you should now be logged into the switch")

    print("Checking the switch for a list of all VLANs")
    all_vlans = Vlan.get_all(s)

    # Checking if VLAN 100 is not in the list of current VLANs, then creating it if necessary
    if (100 not in all_vlans):
        print("VLAN 100 does not exist, adding it to continue.")
        vlanX = Vlan(s, 100, name="VLAN 100", voice=True)
        vlanX.apply()
    
    # Create a VLAN object.  Note that at this point the object is created on the Python instance, and is not yet pushed to the switch
    vlan100 = Vlan(s, 100)
    # Perform a GET request to obtain all data and materialize object
    vlan100.get()
    
    #Now we are able to modify the objects internal attributes
    
    
    if(vlan100.apply()):
        print("VLAN 100 updated")
    else:
        print("VLAN 100 was not successfully updated")
        
    # Create a temporary VLAN object to validate that the switch VLAN 100 has the new settings
    vlanX =
    # Getting the switch VLAN 100 settings and saving it into temporary VLAN
    vlanX.
    
    # Print the voice attribute of vlanX
    print()
    
    
except Exception as error:
        print('Ran into exception: {}.  Closing session.'.format(error))
        
finally:
    print("Now logging out of the switch.")
    s.close()
    

In [ ]:
#### 2.1 VLAN Update Answer Example with notes

import urllib3
from pyaoscx.session import Session
from pyaoscx.pyaoscx_factory import PyaoscxFactory
from pyaoscx.vlan import Vlan
from pyaoscx.interface import Interface

urllib3.disable_warnings()

# Set variables for creating the Session object
version = '10.04'
switch_ip = '16.31.85.161'
username = 'admin'
password = 'Arubacx$'

# Create Session object
s = Session(switch_ip, version)
s.open(username, password)
print("Logging into the switch at ", switch_ip)

try:
    print("By this point you should now be logged into the switch")

    print("Checking the switch for a list of all VLANs")
    all_vlans = Vlan.get_all(s)

    # Checking if VLAN 100 is not in the list of current VLANs, then creating it if necessary
    if (100 not in all_vlans):
        print("VLAN 100 does not exist, adding it to continue.")
        vlanX = Vlan(s, 100, name="VLAN 100", voice=True)
        vlanX.apply()
    
    # Create a VLAN object.  Note that at this point the object is created on the Python instance, and is not yet pushed to the switch
    vlan100 = Vlan(s, 100)
    # Perform a GET request to obtain all data and materialize object
    vlan100.get()
    
    #Now we are able to modify the objects internal attributes
    vlan100.voice = False
    
    if(vlan100.apply()):
        print("VLAN 100 updated")
    else:
        print("VLAN 100 was not successfully updated")
        
    # Create a temporary VLAN object to validate that the switch VLAN 100 has the new settings
    vlanX = Vlan(s, 100)
    # Getting the switch VLAN 100 settings and saving it into temporary VLAN
    vlanX.get()
    
    # Print the voice attribute of vlanX
    print("VLAN 100 Voice {}".format(vlanX.voice))
    
except Exception as error:
        print('Ran into exception: {}.  Closing session.'.format(error))
        
finally:
    print("Now logging out of the switch.")
    s.close()
    

---
The output for this script should be similar to:

```
Logging into the switch at 172.168.23.100
By this point you should now be logged into the switch
Checking the switch for a list of all VLANs
VLAN 100 does not exist, adding it to continue.
VLAN 100 updated
VLAN 100 Voice False
Now logging out of the switch.
```

### Task 2.3 Imperative Factory Approach to Create an Object

* We will Create a VLAN object and validate that it was created using the Imperative Factory Approach

Earlier in Task 1, we importated a library called PyaoscxFactory.  This is used as a way to create a factory of objects that are automatically applied to the switch without the need of using the apply() method.  This can make for smaller code blocks when performing multiple or complex changes.

*Note that when using this approach, if you were to use a factory to "create" VLAN 200 again, it won't error out with a 500 response as the Open Granulated approach did.  However, don't let that stop you from also adding some validation and logic with conditionals as was done previously.*

In [ ]:
#### 2.3 Example with notes

import urllib3
from pyaoscx.session import Session
from pyaoscx.pyaoscx_factory import PyaoscxFactory
from pyaoscx.vlan import Vlan
from pyaoscx.interface import Interface

urllib3.disable_warnings()

# Set variables for creating the Session object
version = '10.04'
switch_ip = '16.31.85.161'
username = 'admin'
password = 'Arubacx$'

# Create Session object
s = Session(switch_ip, version)
s.open(username, password)
print("Logging into the switch at ", switch_ip)

try:
    
    # Getting a list of all VLAN objects on the switch
    print("Checking the switch for a list of all VLANs")
    all_vlans = Vlan.get_all(s)
    print(all_vlans.keys())
    
    # Create a Factory, passing the Session Object.  In this case, the variable name is 'factory'
    factory = PyaoscxFactory(s)
        
    # Create a VLAN object.  If the VLAN is not created yet, the Factory will instantly create it and push that to the switch
    vlan200 = factory.vlan(200, "NAME200")
    
    print("Checking the list of all VLANs for new VLANs")
    all_vlans = Vlan.get_all(s)
    print(all_vlans.keys())
    
except Exception as error:
        print('Ran into exception: {}.  Closing session.'.format(error))
        
finally:
    print("Now logging out of the switch.")
    s.close()
    

---
You should see similar output to:

```
Logging into the switch at 172.168.23.100
Checking the switch for a list of all VLANs
dict_keys([1, 10, 100, 1000, 11, 12, 13, 2, 20, 300, 3000, 4, 400, 6001, 6002, 6003, 6004, 6005, 6006])
Checking the list of all VLANs for new VLANs
dict_keys([1, 10, 100, 1000, 11, 12, 13, 2, 20, 200, 300, 3000, 4, 400, 6001, 6002, 6003, 6004, 6005, 6006])
Now logging out of the switch.
```

As you can see, this method is much shorter and simpler when writing it out.  It is possible to use both methods to interact with each other, so it may be useful to pick and choose different aspects of each approach to create a workflow.

*Note that if you try to run this code multiple times in the Python3 Kernal of Jupyter Notebook, you may see an error `Ran into exception: You cannot create another PyaoscxFactory class.  Closing session.`*
*This is due to the fact that the Factory is still instantiated in the Kernal, so it may be necessary to restart the Kernal in order to run it multiple times.*

### Task 3 Comparison between Approaches with Lag Interfaces

* The first block will show an Open Granulated Approach to add ports to a LAG and tag a VLAN to the lag.
* The second block will show the Imperative Factory Approach in the same example

In [ ]:
#### 3 Example with notes - Open Granulated Approach

import urllib3
from pyaoscx.session import Session
from pyaoscx.pyaoscx_factory import PyaoscxFactory
from pyaoscx.vlan import Vlan
from pyaoscx.interface import Interface

urllib3.disable_warnings()

# Set variables for creating the Session object
version = '10.04'
switch_ip = '16.31.85.161'
username = 'admin'
password = 'Arubacx$'

# Create Session object
s = Session(switch_ip, version)
s.open(username, password)
print("Logging into the switch at ", switch_ip)

try:
    
    print("Checking the switch for a list of all Interfaces")
    all_ints = Interface.get_all(s)
    print(all_ints.keys())    
    
    # Create an Interface object that is a LAG
    lag = Interface(s, 'lag3')
    if ('lag3' not in all_ints):
        # If lag 3 does not exist, adding it to continue.
        lag.apply()
    else:
        lag.get()

    print("Checking the switch for a list of all VLANs")
    all_vlans = Vlan.get_all(s)
    print(all_vlans.keys())

    # Create a Vlan object
    vlan100 = Vlan(s, 100)
    if (100 not in all_vlans):
        # If VLAN 100 does not exist, adding it to continue.
        vlan100.apply()

    vlan100.get()
    # Interfaces/Ports added to LAG
    port_1_1_5 = Interface(s, '1/1/5')
    port_1_1_5.get()
    # Make changes to configure LAG as L2
    lag.admin = 'up'
    lag.routing = False
    lag.vlan_trunks = [vlan100]
    lag.lacp = "passive"
    lag.vlan_mode = "native-untagged"
    lag.vlan_tag = vlan100
    # Add port as LAG member
    if not hasattr(lag, 'interfaces'):
        lag.interfaces = []
    lag.interfaces.append(port_1_1_5)

    # Apply changes
    lag.apply()
    
    lagX = Interface(s, 'lag3')
    lagX.get()
    # Print lagX Interface list of names 
    print("Lag 3 Interfaces:")
    for port in lagX.interfaces:
        print(port.name)
    
except Exception as error:
        print('Ran into exception: {}.  Closing session.'.format(error))
        
finally:
    print("Now logging out of the switch.")
    s.close()
    

---
The above code should have an output similar to:
```
Logging into the switch at 172.168.23.100
Checking the switch for a list of all Interfaces
dict_keys(['1/1/1', '1/1/2', '1/1/3', '1/1/4', '1/1/5', '1/1/6', '1/1/7', '1/1/8', '1/1/9', 'lag2', 'lag3', 'vlan1000', 'vlan3000'])
Checking the switch for a list of all VLANs
dict_keys([1, 10, 100, 1000, 2, 20, 200, 3000, 4, 6001, 6002, 6003, 6004, 6005, 6006])
Lag 3 Interfaces:
1/1/5
Now logging out of the switch.
```
If you have access to the switch CLI, when running a `show running-config` command, the output should have something similar to:
<p align="center">
  <img src="Pictures/OpenLagExample.png">
  
</p>

It may differ depending on the Interfaces, VLANs, and LAGs you may have added while you were testing and coding.

The above block of code used several different combinations of logic that was used in previous examples, and perhaps a few that haven't been used.  Some things to note:
* Using get_all() functions to validate whether there are existing VLANs and Interfaces.
* Using conditionals to determine whether to apply() the object or get() to populate the object with existing data from the switch.
* Directly changing attributes of a LAG.
* Retrieving the new data on the switch with get() and outputting all attributes in a list with a for loop.

The next block of code will execute similar functions to the above with different values, using the Factory Approach


In [ ]:
#### 3 Example with notes - Imperative Factory Approach

import urllib3
from pyaoscx.session import Session
from pyaoscx.pyaoscx_factory import PyaoscxFactory
from pyaoscx.vlan import Vlan
from pyaoscx.interface import Interface

urllib3.disable_warnings()

# Set variables for creating the Session object
version = '10.04'
switch_ip = '16.31.85.161'
username = 'admin'
password = 'Arubacx$'

# Create Session object
s = Session(switch_ip, version)
s.open(username, password)
print("Logging into the switch at ", switch_ip)

try:
    # Create a Factory, passing the Session Object.  In this case, the variable name is 'factory'
    factory = PyaoscxFactory(s)
    # Create the Interface object as a LAG
    lag2 = factory.interface('lag2')
    # Configure the Interface object
    modified = lag2.configure_l2(
        phys_ports=['1/1/4'],
        description="Created using imperative method",
        admin='up',
        vlan_mode="native-untagged",
        vlan_tag=1,
        trunk_allowed_all=True,
        native_vlan_tag=True)
    # If modified is True, a PUT request was done and object was modified

    lagX = Interface(s, 'lag2')
    lagX.get()
    # Print lagX Interface list of names 
    print("Lag 2 Interfaces:")
    for port in lagX.interfaces:
        print(port.name)
    
except Exception as error:
        print('Ran into exception: {}.  Closing session.'.format(error))
        
finally:
    print("Now logging out of the switch.")
    s.close()
    

---
A sample output for the above code is:
```
Logging into the switch at 172.168.23.100
Lag 2 Interfaces:
1/1/4
Now logging out of the switch.
```
If you have access to the switch CLI, when running a `show running-config` command, the output should have something similar to:
<p align="center">
  <img src="Pictures/FactoryLagExample.png">
  
</p>
Due to the Factory method handling existing objects in a simpler fashion, as well as allowing parameters to pass into functions that would configure the interfaces, this is a great way to configure devices with a smaller amount of code.  However, there are use cases where directly interacting with the object attributes can be useful and more efficient as well.  Try out both and find the approach that suits your needs best.


# Lab Summary

Congratulations on completing this session!  To recap, using pyaoscx v2, this workshop taught:

* Setup a Python skeleton framework that allows the user to login and logout of the switch.
* Created VLAN and Interface objects to configure those settings on a switch.
* Used conditionals to validate existing settings and conditionals to handle those use cases.
* Accessed object attributes directly using the Open Granulated Approach
* Used the factory pattern and methods to create and configure settings using the Imperative Factory Approach.
* Compared the methods to evaluate and find the best way for a particular coding style.

Hopefully this was a great initial step into using the pyaoscx v2 Python SDK for Aruba AOS-CX Switches.  
Although the tasks in these workflows were minor, the goal of this automation is to save time and build efficiency for these repetive tasks and scale upwards.
The pyaoscx SDK is constantly being updated, so please keep watch of the public repo [here](https://github.com/aruba/pyaoscx) for more libraries and features in the future.
If you have any suggestions or requests, please use that Github to file issues for our team to help improve the quality and direction of the SDK.
More information can be found on our [Aruba Developer Hub](https://developer.arubanetworks.com/)


# Conclusion
This session is intended to get you started with the pyaoscx v2 SDK.  This SDK functions as the foundation for other Aruba Automation frameworks and tools, such as Ansible and NAPALM.
For those interested in taking their Aruba automation journey further, please reach out to our Aruba Automation PDL at [aruba-automation@hpe.com](mailto:aruba-automation@hpe.com)

* Let's conclude [here](Workshop-on-Demand-Conclusion.ipynb)